# Teste do Software de um formulário em um site

## Planejamento
- risco: baixo, risco de insersao de tuplas no BD
- recursos: selenium webdriver, python notebook, oraculo
- ambiente: ipynb, github, chrome, website: https://itera-qa.azurewebsites.net/home/automation
- criterio de parada: o codigo passar em todos os testes, nao havendo tuplas problemáticas
- resultados esperados: esperamos, como relatado da equipe de desenvolvimento, problemas da verificação do formulário
    


In [1]:
from selenium import webdriver #para abrir navegador
from selenium.webdriver.common.by import By #para buscar elementos html por ID
from selenium.webdriver.support.ui import WebDriverWait #para esperar a conexao
from selenium.webdriver.support import expected_conditions as EC #para verificar possiveis erros no front
import time #sleep

_url_site = 'https://itera-qa.azurewebsites.net/home/automation'

_request = {
    'nome':'',
    'numero':'',
    'email':'',
    'senha':'',
    'endereco':''
}

## Projeto de casos de teste
- definir cenários: cenário de erros na insercao e cenário de tentativa de invasão de conta, no formulário
- implementar teste: será implementado em python, notebook, usando web-driver e salvando os resultados em csvs com o pandas
- priorizar casos: priorizar casos de tentativa de invasão


### Tuplas que podem dar problemas
Seguindo como o Software deve ser implementado os requisitos relacionados a este formulário são:
- nome: 2 a 25 caracteres, letras e acentuacao permitido
- telefone: permitidos seguintes estruturas (xx)xxxxx-xxxx, xxxxxxxxxxx
- email: 8 a 25 caracteres xx@xx.xx
- senha: 4 a 25 caracteres contendo pelo menos 1 numero 1 caractere especial 1 uppercase
- endereco: 4 a 60 caracteres

Criando funcoes que sao capazes de cherar casos de testes aleatorios

In [2]:
#gerar tuplas que podem dar problema
import re
import os
import string
import random

#baseado nos testes do Professor Paulo Matias
def random_letters(n):
    res = b''
    while len(res) < n: 
        res += re.sub(br'[^a-z]', b'', os.urandom(512))
    return str(res[:n])[2:-1]

def random_numbers(length):
    digits = string.digits
    return ''.join(random.choice(digits) for _ in range(length))

def random_special_characters_string(length):
    special_characters = string.punctuation
    return ''.join(random.choice(special_characters) for _ in range(length))

def gerar_tupla_consistente():
    _request['nome'] = random_letters(random.randint(2,25))
    _request['numero'] = '('+random_numbers(2)+')'+random_numbers(5)+'-'+random_numbers(4)
    _request['email'] = random_letters(random.randint(2,10))+'@'+random_letters(random.randint(2,10))+'.'+random_letters(3)
    _request['senha'] = random_letters(random.randint(1,10))+random_numbers(random.randint(1,10))+random_special_characters_string(random.randint(1,5))
    _request['endereco'] = random_letters(random.randint(4,60))

def gerar_tupla_inconsistente():
    
    _request['nome'] = random.choice([random_letters(random.choice([26,1])),random_numbers(10),random_special_characters_string(10)])
    _request['numero'] = random.choice([random_letters(9),random_special_characters_string(9),random_numbers(random.choice([8,10]))])
    _request['email'] = random_letters(random.randint(2,10))+'.'+random_letters(random.randint(2,10))+'@'+random_letters(3)
    _request['senha'] = random_numbers(random.randint(1,10))+random_special_characters_string(random.randint(1,5))
    _request['endereco'] = random_letters(61)



In [3]:

import Back_End_Simulator as BE



def Teste_De_Formulario():

    validador = BE.Validador()
    # Configuração do WebDriver
    driver = webdriver.Chrome()  # Certifique-se de ter o ChromeDriver configurado e no PATH
    driver.get(_url_site)

    try:
        # Preencher o campo Name
        
        name_field = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'name')))
        
        name_field.send_keys(_request['nome'])

        #Preencher o campo Mobile number
        mobile_field = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'phone')))
        mobile_field.send_keys(_request['numero'])

        # Preencher o campo Email address
        email_field = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'email')))
        email_field.send_keys( _request['email'])

        # Preencher o campo Password
        password_field = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'password')))
        password_field.send_keys(_request['senha'])

        # Preencher o campo Address
        address_field = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'address')))
        address_field.send_keys(_request['endereco'])

        # Clicar no botão Submit
        submit_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[type="submit"]')))
        time.sleep(0.01)
        submit_button.click()

        print('Formulário preenchido e enviado com sucesso!')
        response = validador.validar_request(_request)
        
        return False,'',response

    except Exception as e:
        print('Ocorreu um erro durante o preenchimento do formulário:', str(e))

        return False,'Entrada Inconsistente',response

    finally:
        driver.quit()  # Fechar o navegador após a conclusão do teste



## Configuração de ambiente
- teste do ambiente e ferramentas: configurar todas as ferramentas que serão usadas e deixar claro como usa-las


In [4]:
#certificar de usar o webdriver do chrome
#coloca-lo no path
#rodar todos os codigos acima, devem estar sem erros


## Execução e Avaliação
- executar teste:
- avaliar teste:


### Teste com entradas corretas e consistentes

In [5]:
#testar o codigo
def classificar_respostas():
    resultado,resposta,lista_erro = Teste_De_Formulario()
    print(resposta)

    dict_erro = {}
    for i in range(0, len(lista_erro), 2):
        chave = lista_erro[i]
        valor = lista_erro[i + 1]
        dict_erro[chave] = valor
    #nesse caso o oraculo eh que todos devem ser positivos, sem inconsistencias
    if not resultado:
        print('Falhou no teste!')
        print(dict_erro)

    return dict_erro

In [6]:
#gerar entradas
gerar_tupla_consistente()
#testar
classificar_respostas()

Formulário preenchido e enviado com sucesso!
Há dados invalidos!:
['numero:', '(46)91686-0464']

Falhou no teste!
{'numero:': '(46)91686-0464'}


{'numero:': '(46)91686-0464'}

In [7]:

#quebrar os dados para que todos sejam inconsistentes
gerar_tupla_inconsistente()
#testar
classificar_respostas()


Formulário preenchido e enviado com sucesso!
Ocorreu um erro durante o preenchimento do formulário: 'bool' object is not subscriptable


UnboundLocalError: cannot access local variable 'response' where it is not associated with a value

In [8]:
erro_nome     = 0
erro_email    = 0
erro_numero   = 0
erro_senha    = 0
erro_endereco = 0

#bateria de testes
for i in range(1,4):
    if i<=1:
        gerar_tupla_consistente()
    else:
        gerar_tupla_inconsistente()

    try: 
        erros = classificar_respostas()
    except:
        print('erro durante o teste')

    if 'nome:' in erros:
        erro_nome += 1

    if 'email:'in erros:
        erro_email+= 1

    if 'numero:'in erros:
        erro_numero+=1

    if 'senha:'in erros:
        erro_senha+=1

    if 'endereco:'in erros:
        erro_endereco+= 1

    print('-----------------------------')
    print('')

print('Total de Erros:')
print('nome:'+str(erro_nome))
print('email:'+str(erro_email))
print('numero:'+str(erro_numero))
print('senha:'+str(erro_senha))
print('endereco:'+str(erro_endereco))

Formulário preenchido e enviado com sucesso!
Há dados invalidos!:
['numero:', '(59)78574-1852']

Falhou no teste!
{'numero:': '(59)78574-1852'}
-----------------------------

Formulário preenchido e enviado com sucesso!
Há dados invalidos!:
['nome:', '9721877680', 'email:', 'pdmeuscnyd.bgahnqd@tbg', 'numero:', '{.%:<|^_|', 'endereco:', 'owwgtzqgujqktigplolyeguvwudsoknuuofhlkebsecmzdznskwjtwvfcrpth']

Falhou no teste!
{'nome:': '9721877680', 'email:': 'pdmeuscnyd.bgahnqd@tbg', 'numero:': '{.%:<|^_|', 'endereco:': 'owwgtzqgujqktigplolyeguvwudsoknuuofhlkebsecmzdznskwjtwvfcrpth'}
-----------------------------

Formulário preenchido e enviado com sucesso!
Ocorreu um erro durante o preenchimento do formulário: 'bool' object is not subscriptable
erro durante o teste
-----------------------------

Total de Erros:
nome:2
email:2
numero:3
senha:0
endereco:2



## Monitoramento e Controle
- checar status:
- verificar novos possíveis testes:
- verificar se corrigiram o que estava errado:

In [17]:
#evitar bots
#validar codigo no front
#corrigir erro no telefone no caso consistente
#corrigir erro na senha no caso inconsistente
#sanitizar entradas para o banco de dados
#garantir seguranca das senhas
#refatorar codigo de teste para ser mais maduro e consistente